<a href="https://colab.research.google.com/github/fatemafaria142/Analyzing-Large-Language-Models-with-LangChain/blob/main/Mental_Health_Advice_Generation_using_openchat_3_5_1210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [4]:
mode_id = "openchat/openchat-3.5-1210"

In [5]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
* https://python.langchain.com/docs/modules/memory/types/buffer


### **1. ConversationBufferMemory:**
* This memory allows for storing messages and then extracts the messages in a variable.
* Memory maintains Chain state, incorporating context from past runs.

* Create a new model by parsing and validating input data from keyword arguments.

* Raises ValidationError if the input data cannot be parsed to form a valid model.

In [20]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_k = 50  # You can adjust this value based on your preference
top_p = 0.95  # You can adjust this value based on your preference
temperature = 0.8  # You can adjust this value based on your preference

pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_k=top_k, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)


#Creating a Mental Health Support Prompt Using LangChain
template = """GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: {context}<|end_of_turn|>
GPT4 Correct Assistant: """
prompt = PromptTemplate(template=template, input_variables=["context"])

memory = ConversationBufferMemory(human_prefix="User", ai_prefix="Mental Health Counselor")

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,  #(setting verbose=True so we can see the prompt
    memory=memory
)

### **Example: 01**

In [28]:
context = "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?"
conversation.predict(context=input)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on.<|end_of_turn|>
GPT4 Correct Assistant: 

> Finished chain.


"1. I'm really sorry to hear about the challenges you and your family are facing. It's normal to feel scared and overwhelmed in this situation. It's important to remember that you are not alone and that it's okay to ask for help.\n\n2. It's understandable that your biological daughter might be blaming you for her step-sibling's overdose. It's important to acknowledge her feelings, but also to remind her that she is not responsible for her step-sibling's choices. Encourage her to express her feelings and concerns to you, and to seek support if needed.\n\n3. It's also important to acknowledge your own feelings of fear and blame. You may want to consider seeking individual counseling to help you process these emotions and gain strategies for managing them.\n\n4. It's crucial to maintain open lines of communication with all of your children. Encourage them to share their thoughts and feelings with you, and let them know that you are there to support them. Remember, you cannot control their

### **Example: 02**

In [29]:
input = "I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on."
conversation.predict(context=input)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have three children: a biological daughter, a stepdaughter, and a stepson. I found out my stepdaughter overdosed today. My daughter blames me. I told her to stay out of it. I'm scared of all three kids now. They all blame me for their choices they made. All three have drug addictions. I had a stroke in 2013 so I need to depend on people from now on.<|end_of_turn|>
GPT4 Correct Assistant: 

> Finished chain.


'İşte, arkadaş, ben de sizi, sizinle ilgili herhangi bir durumda ve her zaman saygı gerektiren ve gerektiği gibi değerli olan bir güvence olarak davet ediyorum. Bu süreçte, sizinle yapıcı ve anlayıcı bir ortam sağlamamam gibi düşünmek büyük bir çekim olsa da, sizinle olanınız hakkında anlamanız gerekmektedir.\n\nÖncelikle, fikirinizi kendinize dikkatle ele alınca, kendinize ve hala hareket ediyor olup olmadığınıza dikkat edin. Kendinize olanınız hakkında anlamanız önemlidir ve kendinize olanınız hakkında dikkatlice bir dikkat etmek, sizinle ilgili herhangi bir durumda olsa önemlidir.\n\nYakın ilişkilerinizde yaşadığınız türde anlaşmazlıklar ve endişeler, gönüllü bir destek sağlayıcının da olduğu gibi, bir uzmanın da olduğu gibi, uzun süre çözülemez olabilir. Ancak, her zaman, herhangi bir durumda dikkatli ve hoşgörülü bir şekilde davranarak, daha iyi hale gelmenize ve bu anlaşmazlıkları çözmenize yardımcı olabiliriz.\n\nYakın ilişkilerinizde yaşadığınız endişeleri ele alarak, bir uzman

### **Example: 03**

In [30]:
input = "I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?"
conversation.predict(context=input)



> Entering new LLMChain chain...
Prompt after formatting:
GPT4 Correct User: Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: I have a friend that who I used to be in a relationship with. It was brief and turned into us being just good friends. I spent the weekend with him and  it upset my boyfriend. Was i wrong?<|end_of_turn|>
GPT4 Correct Assistant: 

> Finished chain.


"\n\nDear [Friend],\n\nI'm here to offer you understanding and guidance. It seems like you're dealing with a situation where your boyfriend was upset because you spent the weekend with an ex-partner who is now a good friend. It's natural to feel confused and unsure about how to handle this situation. Let's explore this together.\n\nFirst, it's important to acknowledge your feelings. It's normal to feel a sense of guilt or concern when your actions upset someone you care about. This is a sign that you value your relationship with your boyfriend and want to maintain a healthy connection.\n\nNext, let's consider the context of your relationship with your ex-partner. If your relationship turned into a friendship after ending the romantic aspect, it's essential to understand that friendships can be just as valuable and meaningful as romantic relationships. As long as your friendship with your ex is respectful and doesn't involve any romantic feelings or actions, it can be a healthy part of 

In [31]:
#We can also get the history as a list of messages (this is useful if you are using this with a chat model).
memory.load_memory_variables({})

{'history': "User: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?\nMental Health Counselor: İ am so sorry to hear that you are struggling with these intense feelings. It must be really difficult for you to go through this alone. First, I want to make it clear that it is not your fault that you feel this way. Sometimes, our brains can be really unkind to us, but they are not always accurate in how they see things. \n\nYou mentioned that you barely sleep and are constantly thinking about your worthlessness. It can be really hard to break this cycle on your own. It might be helpful to reach out to someone who can help you, like a mental health professional or a trusted person in your life. Talking to someone can h

In [32]:
print(memory.buffer)

User: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it. How can I change my feeling of being worthless to everyone?
Mental Health Counselor: İ am so sorry to hear that you are struggling with these intense feelings. It must be really difficult for you to go through this alone. First, I want to make it clear that it is not your fault that you feel this way. Sometimes, our brains can be really unkind to us, but they are not always accurate in how they see things. 

You mentioned that you barely sleep and are constantly thinking about your worthlessness. It can be really hard to break this cycle on your own. It might be helpful to reach out to someone who can help you, like a mental health professional or a trusted person in your life. Talking to someone can help you to see th